# ベイズ勉強会 Part 7 ポアソン分布

> ポアソン分布のベイズ推論

- toc: true 
- badges: true
- comments: true
- categories: [bayes]
- image: images/dag1.png

ベイズ勉強会資料は『ベイズ推論による機械学習入門』{% fn 1 %}を元に、途中式計算をできるだけ省略せずに解説を行ったものです。

## ポアソン分布

ポアソン分布は次のような確率密度関数を分布関数にもつ確率分布である。

$$
\mathrm{Poi} (x|\lambda) = \frac{\lambda^x}{x!} \mathrm{e}^{-\lambda}
$$

xは非負の整数、パラメータ$\lambda$は正の実数である。

対数をとると

$$
\ln \mathrm{Poi} (x|\lambda) = x \ln \lambda - \ln x! - \lambda
$$

である。

## ガンマ分布

ポアソン分布のパラメータ$\lambda$は正の実数であるので、事前分布$p(\lambda)$はガンマ分布で表すと都合が良い。ガンマ分布は次のような確率密度関数を分布関数にもつ確率分布である。

$$
\mathrm{Gam}(\lambda|a,b) = C_G(a,b)\lambda^{a-1} \mathrm{e}^{-b\lambda}
$$

正規化係数$C_G(a,b)$は次のような関数である。

$$
C_G(a,b) = \frac{b^a}{\Gamma(a)}
$$

ガンマ分布の対数をとると

$$
\ln \mathrm{Gam}(\lambda|a,b) = (a-1)\ln \lambda - b\lambda + \ln C_G(a,b)
$$

である。

## モデル構築

ある池で1時間釣りをした時に釣れる魚の数をベイズ推論で予測してみる。1時間の釣りをN回試した時の釣れた魚の数を${\bf X} = {x_1,\dots,x_N}$とすると、予測のためのモデルは次のように書ける。

$$
\begin{eqnarray}
    p({\bf X},\lambda) &=& p({\bf X}|\lambda)p(\lambda) \\
    p({\bf X}|\lambda) &=& \Pi_{n=1}^N \mathrm{Poi}(x_n|\lambda) \\
    p(\lambda) &=& \mathrm{Gam}(\lambda|a,b)
\end{eqnarray}
$$

事前分布のパラメータ$a,b$はハイパーパラメータとなる。

## 事後分布の推論

事後分布$p(\lambda|{\bf X})$は同時分布を観測されたデータ${\bf X}$の確率分布で割れば求まる。ただし${\bf X}$は$\lambda$を含んでおらず対数をとることにより定数項として扱える。

$$
\begin{eqnarray}
    \ln p(\lambda|{\bf X}) &=& \ln p({\bf X}|\lambda)p(\lambda) + const. \\
    &=& \ln \Pi_{n=1}^N \mathrm{Poi}(x_n|\lambda) + \ln \mathrm{Gam}(\lambda|a,b) + const. \\
    &=& \Sigma_{n=1}^N \{ x_n \ln \lambda - \ln x_n ! - \lambda \} + (a-1)\ln \lambda - b\lambda + \ln C_G(a,b) + cosnt. \\
    &=& (\Sigma_{n=1}^N x_n + a - 1) \ln \lambda - (N + b) \lambda + const.
\end{eqnarray}
$$

これはガンマ分布の対数をとった形である。よって事後分布は次のように表せる。

$$
\begin{eqnarray}
    p(\lambda|{\bf X}) &=& \mathrm{Gam}(\hat{a},\hat{b}) \\
    ただし　\hat{a} &=& \Sigma_{n=1}^N x_n + a \\
    \hat{b} &=& N + b
\end{eqnarray}
$$

## 予測分布の導出

パラメータ$\lambda$の事前分布と事後分布の形状が同じポアソン分布となるので、事前分布を用いて事前予測分布を求めた後、学習済みのハイパーパラメータを代入して学習済み予測分布を求める。未観測の変数$x_*$は観測${\bf X}$と同じ分布から独立に生成されるとして、予測分布$p(x_*)$は次のように求まる。

$$
\begin{eqnarray}
    p(x_*) &=& \int p(x_*|\lambda)p(\lambda)d\lambda \\
    &=& \int \mathrm{Poi}(x_*|\lambda)\mathrm{Gam}(\lambda|a,b)d\lambda \\
    &=& \int \frac{\lambda^{x_*} }{x_* !} \mathrm{e}^{-\lambda} C_G(a,b) \lambda^{a-1} \mathrm{e}^{-b\lambda} d\lambda \\
    &=& \frac{C_G(a,b)}{x_* !} \int \lambda^{x_* + a - 1} \mathrm{e}^{-(1+b)\lambda} d\lambda \\
    &=& \frac{C_G(a,b)}{x_* ! C_G(x_* + a, 1+b)}　(ガンマ分布の定義式より積分部分はガンマ分布の正規化係数の形で表せる) \\
    &=& \frac{b^a \Gamma(x_* + a)}{x_* ! \Gamma(a) (1+b)^{x_* + a} } \\
    &=& \frac{\Gamma(x_* + a)}{x_* ! \Gamma(a)} (\frac{b}{1+b})^{a} (\frac{1}{1+b})^{x_*}
\end{eqnarray}
$$

$r = a, p = \frac{1}{b+1}$とおくと、

$$
p(x_*) = \frac{\Gamma(x_* + r)}{x_* ! \Gamma(r)} (1-p)^r p^{x_*}
$$

となり、これは負の二項分布(negative binomial distribution)の形である。よって$p(x_*)$は次のように表せる。

$$
\begin{eqnarray}
    p(x_*) &=& \mathrm{NB}(x_*|r,p) \\
    &=& \frac{\Gamma(x_* + r)}{x_* ! \Gamma(r)} (1-p)^r p^{x_*} \\
    ただし　r &=& a \\
    p &=& \frac{1}{b+1}
\end{eqnarray}
$$

$a,b$を$\hat{a},\hat{b}$に置き換えれば学習後の予測分布が求まる。

{{ '[須山敦志. 杉山将. ベイズ推論による機械学習入門. 講談社, 2017.](https://www.kspub.co.jp/book/detail/1538320.html)' | fndetail: 1 }}